In [1]:
#Import basic python packages for data analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.lines as mlines
import pylab as plot
import matplotlib
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import time
import os

import torch

## Pytorch practice

In [31]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data

shape = (2,3,)
rand_tensor = torch.rand(shape)
rand_tensor

tensor = torch.ones(4, 4)
tensor.shape

t1 = torch.cat([tensor, tensor, tensor], dim=-2)

t = torch.ones(5)
t

n = t.numpy()
print(f"n: {n}")

t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

n: [1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


In [32]:
t.sum()

tensor(10.)

## (A) Dataset prep

In [21]:
# merge all datasets to create master_df

root = os.path.dirname(os.path.dirname(os.getcwd()))

df_aq = pd.read_csv(root + "/data/cleaned/air_quality_NO2.csv", index_col=0)[['value','latitude', 'longitude']]
df_met = pd.read_csv(root + "/data/cleaned/nO2_met.csv", index_col=0)
df_fac = pd.read_csv(root + "/data/cleaned/no2_fac_data.csv", index_col=0)
# df_fac.drop(df_fac.columns[df_fac.columns.str.contains('_emsdist')], axis=1, inplace=True)
df_traffic = pd.read_csv(root + "/data/cleaned/intersection_final.csv", index_col=0)

df_m1 = df_aq.merge(df_met, on = ['latitude', 'longitude'], how = 'inner')
df_m2 = df_m1.merge(df_fac, on = ['latitude', 'longitude'], how = 'inner')
df_merged = df_m2.merge(df_traffic, on = ['latitude', 'longitude'], how = 'inner')
df_merged.drop(columns = ['latitude', 'longitude'], inplace=True)

X = df_merged.drop("value",1) 
y = df_merged["value"]

In [22]:
# function to scale and transform input data
# def get_data(X, y):
#     X = X.values
#     y = y.values
#     # scaling the data
#     feature_scaler = StandardScaler()
#     X = feature_scaler.fit_transform(X)
#     return X, y

# # acquiring transformed data
# X_arr, y_arr = get_data(X, y)

# # splitting into test and train
# X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.3)
# cols = np.array(X.columns)

In [40]:
# splitting into test and train
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3)
cols = np.array(X.columns)

## (B) Modeling

In [41]:
# dataset class for feeding in data
class AirQualityDataset(torch.utils.data.Dataset):

  def __init__(self, X_arr, y_arr):
    self.x_data = torch.tensor(X_arr, \
      dtype=torch.float32)
    self.y_data = torch.tensor(y_arr, \
      dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    preds = self.x_data[idx,:]  # or just [idx]
    conc = self.y_data[idx] 
    return (preds, conc)       # tuple of matrices

# prepping data for training
batch_size = 64
train_ds = AirQualityDataset(X_train, y_train)
test_ds = AirQualityDataset(X_test, y_test)
train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_ldr = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [42]:
# network architecture
# class NeuralNetwork(torch.nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()
#         self.linear_relu_stack = torch.nn.Sequential(
#             torch.nn.Linear(82, 30),
#             torch.nn.ReLU(),
#             torch.nn.Linear(30, 30),
#             torch.nn.ReLU(),
#             torch.nn.Linear(30, 1),
#         )

#     def forward(self, x):
#         pred_conc = self.linear_relu_stack(x)
#         return pred_conc

In [43]:
# network architecture
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(82, 40),
            torch.nn.Tanh(),
            torch.nn.Linear(40, 30),
            torch.nn.Tanh(),
            torch.nn.Dropout(p=0.2)
            torch.nn.Linear(30, 20),
            torch.nn.Tanh(),
            torch.nn.Linear(20, 20),
            torch.nn.Tanh(),
            torch.nn.Linear(20, 1),
        )

    def forward(self, x):
        pred_conc = self.linear_relu_stack(x)
        return pred_conc

SyntaxError: invalid syntax (<ipython-input-43-a89d270dedf8>, line 11)

In [44]:
# creating model instance
model = NeuralNetwork()
print(model)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
    
# initialising hyperparameters
learning_rate = 1e-2
epochs = 10

# initializing the loss function
loss_fn = torch.nn.MSELoss()

# initializing the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=82, out_features=40, bias=True)
    (1): Tanh()
    (2): Linear(in_features=40, out_features=30, bias=True)
    (3): Tanh()
    (4): Linear(in_features=30, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=1, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([40, 82]) | Values : tensor([[-5.2547e-02, -8.4424e-02,  8.0228e-02, -3.0330e-02,  9.3842e-02,
         -7.8985e-02,  3.3429e-02,  5.7034e-02, -4.7744e-02,  1.7889e-02,
         -3.3550e-02,  6.9205e-02,  9.0955e-02,  6.7328e-02,  1.0035e-01,
          2.9098e-02,  1.0720e-01,  1.0054e-01, -7.0013e-02, -2.1861e-02,
          9.0723e-02, -9.6244e-02, -1.0984e-01, -6.0199e-02,  5.7629e-02,
         -8.2884e-02,  7.0488e-02,  1.0986e-01, -1.0832e-01, -4.3913e-02,
          3.4585e-02, -3.2573e-02, -3.9653e-02,  4.1680e-02, -9

In [45]:
# defining train and test loops

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [46]:
# executing training and testing
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_ldr, model, loss_fn, optimizer)
    test_loop(test_ldr, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.119066  [    0/ 7511]
loss: 0.000003  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 2
-------------------------------
loss: 0.000001  [    0/ 7511]
loss: 0.000001  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 3
-------------------------------
loss: 0.000006  [    0/ 7511]
loss: 0.000000  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 4
-------------------------------
loss: 0.000000  [    0/ 7511]
loss: 0.000000  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 5
-------------------------------
loss: 0.000004  [    0/ 7511]
loss: 0.000001  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 6
-------------------------------
loss: 0.000001  [    0/ 7511]
loss: 0.000009  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 7
-------------------------------
loss: 0.000001  [    0/ 7511]
loss: 0.000000  [ 6400/ 7511]
Test Error: Avg loss: 0.000002 

Epoch 8
-------------------------------
loss: 0.000001  [    0/ 7511]